<a href="https://colab.research.google.com/github/PrithviRaj037/PyTorchProjects/blob/main/PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install opendatasets --quiet
import opendatasets as od
od.download("https://www.kaggle.com/datasets/mssmartypants/rice-type-classification")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: Pirthvi raj
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/mssmartypants/rice-type-classification


100%|██████████| 888k/888k [00:00<00:00, 740MB/s]

In [2]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)


cuda


In [3]:
data_df = pd.read_csv("/content/rice-type-classification/riceClassification.csv")
data_df.head()

,id,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation,Class
0,1,4537,92.229316,64.012769,0.719916,4677,76.004525,0.657536,273.085,0.764510,1.440796,1
1,2,2872,74.691881,51.400454,0.725553,3015,60.471018,0.713009,208.317,0.831658,1.453137,1
2,3,3048,76.293164,52.043491,0.731211,3132,62.296341,0.759153,210.012,0.868434,1.465950,1
3,4,3073,77.033628,51.928487,0.738639,3157,62.551300,0.783529,210.657,0.870203,1.483456,1
4,5,3693,85.124785,56.374021,0.749282,3802,68.571668,0.769375,230.332,0.874743,1.510000,1


In [4]:
data_df.dropna(inplace=True) # frop missing values
data_df.drop(['id'], axis = 1, inplace = True) # drop id column
data_df.head()

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation,Class
0,4537,92.229316,64.012769,0.719916,4677,76.004525,0.657536,273.085,0.764510,1.440796,1
1,2872,74.691881,51.400454,0.725553,3015,60.471018,0.713009,208.317,0.831658,1.453137,1
2,3048,76.293164,52.043491,0.731211,3132,62.296341,0.759153,210.012,0.868434,1.465950,1
3,3073,77.033628,51.928487,0.738639,3157,62.551300,0.783529,210.657,0.870203,1.483456,1
4,3693,85.124785,56.374021,0.749282,3802,68.571668,0.769375,230.332,0.874743,1.510000,1


In [5]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=data_df)

MessageError: Error: credential propagation was unsuccessful

In [6]:
print (data_df["Class"].unique())

[1 0]


In [7]:
print(data_df["Class"].value_counts())

Class
1    9985
0    8200
Name: count, dtype: int64


In [8]:
original_df = data_df.copy()
for column in data_df.columns:
  data_df[column] = data_df[column] / data_df[column].abs().max()
data_df.head()

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation,Class
0,0.444368,0.503404,0.775435,0.744658,0.424873,0.666610,0.741661,0.537029,0.844997,0.368316,1.0
1,0.281293,0.407681,0.622653,0.750489,0.273892,0.530370,0.804230,0.409661,0.919215,0.371471,1.0
2,0.298531,0.416421,0.630442,0.756341,0.284520,0.546380,0.856278,0.412994,0.959862,0.374747,1.0
3,0.300979,0.420463,0.629049,0.764024,0.286791,0.548616,0.883772,0.414262,0.961818,0.379222,1.0
4,0.361704,0.464626,0.682901,0.775033,0.345385,0.601418,0.867808,0.452954,0.966836,0.386007,1.0


In [9]:
X = np.array(data_df.iloc [:,:-1])
Y = np.array(data_df.iloc [:,-1])


In [10]:
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size= 0.3)

In [11]:
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size = 0.5)

In [12]:
print(X_train.shape, X_test.shape, X_val.shape)

(12729, 10) (2728, 10) (2728, 10)


In [15]:
class dataset(Dataset):
  def __init__(self, X, Y):
    self.X = torch.tensor(X, dtype = torch.float32).to(device)
    self.Y = torch.tensor(Y, dtype = torch.float32).to(device)

  def __len__(self):
    return len(self.X)

  def __getitem__(self, idx):
    return self.X[idx], self.Y[idx]

In [16]:
training_data = dataset(X_train, y_train)
validation_data = dataset(X_val, y_val)
testing_data = dataset(X_test, y_test)

In [17]:
train_dataloader = DataLoader(training_data, batch_size = 8, shuffle = True)
validation_dataloader = DataLoader(validation_data, batch_size = 8, shuffle = True)
test_dataloader = DataLoader(testing_data, batch_size = 8, shuffle = True)

In [18]:
for x, y in train_dataloader:
  print(x)
  print("======")
  print(y)
  break

tensor([[0.6410, 0.7514, 0.7431, 0.9260, 0.6067, 0.8006, 0.8834, 0.6396, 0.8594,
         0.5737],
        [0.6552, 0.7788, 0.7382, 0.9353, 0.6240, 0.8095, 0.7329, 0.6691, 0.8026,
         0.5985],
        [0.6696, 0.8713, 0.6683, 0.9706, 0.6340, 0.8183, 0.5728, 0.6959, 0.7583,
         0.7397],
        [0.8982, 0.9023, 0.8657, 0.9327, 0.8505, 0.9478, 0.6168, 0.7681, 0.8350,
         0.5913],
        [0.4897, 0.7306, 0.5925, 0.9628, 0.4697, 0.6998, 0.5937, 0.6032, 0.7382,
         0.6995],
        [0.6052, 0.7919, 0.6791, 0.9540, 0.5833, 0.7779, 0.6419, 0.6625, 0.7561,
         0.6616],
        [0.5461, 0.6912, 0.6903, 0.9237, 0.5168, 0.7390, 0.6428, 0.5942, 0.8482,
         0.5681],
        [0.9456, 0.8899, 0.9228, 0.9145, 0.8940, 0.9724, 0.9019, 0.7735, 0.8668,
         0.5471]], device='cuda:0')
tensor([0., 0., 1., 0., 1., 1., 1., 0.], device='cuda:0')


In [22]:
HIDDEN_NEURONS = 10
class MyModel(nn.Module):
  def __init__(self):
    super(MyModel, self).__init__()

    self.input_layer = nn.Linear(X.shape[1], HIDDEN_NEURONS)
    self.linear = nn.Linear(HIDDEN_NEURONS, 1)
    self.sigmoid = nn.Sigmoid()
  def forward(self, x):
    x = self.input_layer(x)
    x = self.linear(x)
    x = self.sigmoid(x)
    return x

model = MyModel().to(device)

In [23]:
summary(model, (X.shape[1],))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 10]             110
            Linear-2                    [-1, 1]              11
           Sigmoid-3                    [-1, 1]               0
Total params: 121
Trainable params: 121
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


In [25]:
criterion = nn.BCELoss()
optimizer = Adam(model.parameters(), lr = 0.001)

In [29]:
total_loss_train_plot = []
total_loss_validation_plot = []
total_acc_train_plot = []
total_acc_validation_plot = []

epochs = 10
for epoch in range(epochs):
  total_acc_train = 0
  total_loss_train = 0
  total_acc_val = 0
  total_loss_val = 0

  for data in train_dataloader:
    inputs, labels = data

    prediction = model(inputs)
    print(prediction.shape)
    print(prediction.squeeze(1).shape)
    #break
    batch_loss = criterion(prediction, labels)

    total_loss_train += batch_loss.item()







torch.Size([8, 1])
torch.Size([8])
torch.Size([8, 1])
torch.Size([8])
torch.Size([8, 1])
torch.Size([8])
torch.Size([8, 1])
torch.Size([8])
torch.Size([8, 1])
torch.Size([8])
torch.Size([8, 1])
torch.Size([8])
torch.Size([8, 1])
torch.Size([8])
torch.Size([8, 1])
torch.Size([8])
torch.Size([8, 1])
torch.Size([8])
torch.Size([8, 1])
torch.Size([8])
